# Ecological Correlates of Physical Access to Skilled Nursing Facilities in the United States: Findings From the 2020 Release of The American Hospital Association Database

## Introduction

- Evidence suggests that aged patients that receive care from medical professionals at nursing home report better health outcomes relative to those receiving care from other providers.
    - [https://aspe.hhs.gov/basic-report/physician-practices-nursing-homes-final-report](https://aspe.hhs.gov/basic-report/physician-practices-nursing-homes-final-report)
- Moreover, aged patients that receive care in skilled nursing facilities are less likely to receive fragmented care, as they typically receive care in a single facility, which decreases the odds of forms of treatment with contraindications.
    - [https://jamanetwork.com/journals/jamainternalmedicine/article-abstract/2643345](https://jamanetwork.com/journals/jamainternalmedicine/article-abstract/2643345)
- It is well-documented that family involvement in long-term care, and visitations are positively correlated with improved mental health among aged-patients at skilled nursing facilities.
    - [https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2247412/](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2247412/)
- Given these considerations, having skilled nursing facilities in proximity of prospective patients' existing communities has promise in promoting the physical and mental health of aged citizens in the US.
- To date, there is limited literature on the spatial distribution of skilled nursing facilities in the US.
- The objective of this study is examine physical access to skilled nursing facilities in th US among individuals aged 65 years and older.

## Methods

- Data on the location and attributes of hospitals in the US comes from the American Hospital Association Database.
    - https://www.ahadata.com/ 
    - Identification of skilled nursing facilities occurs through the column \[SNHOS\], which comes from the table \[dbo\].\[as19scv1\]
- Census Tract aggregated population level data comes from the 2014-2018 American Community Survey 5-Year Estimates
    - https://www.census.gov/geographies/mapping-files/time-series/geo/tiger-data.html
    - Population counts among individuals aged 65 years and older were derived from this data source
- The socioeconomic status of the population's census tracts were operationalized using the Area Deprivation Index (ADI)
    - https://www.neighborhoodatlas.medicine.wisc.edu/
    - In this data source higher scores reflect higher odds of deprivation
    - The ADI was categorized into three groups, which go as follows:
        - 0-25 high SES
        - 25-75 middle SES
        - 76+ low SES

- Rurality was operationalized using HRSA's Rural Health Area
    - https://data.hrsa.gov/topics/rural-health
    - This data source categorizes a census tract as rural and non-rural.
- ESRI's StreetMap Premium was used to estimate morning weekday travel time between the geometric centroid of each census tract in the US and the skilled nursing facility.
- In this analysis, we examined 1) the number of skilled nursing facilities within an hour travel time, and 2) the travel time to the closest skilled nursing facility for each census tract, respectively.
- Comparisons were made by SES and rurality within each state in the US.

In [1]:
Use HCAN361_FinalProj

--Step 1: Select the ID's of skilled nursing hospitals
select ID
into #key_ID
from dbo.as19svc1
where SNHOS=1;
--744 of these hospitals

--Step 2: Find target population of interest for these services
--men 65+ --> B01001e20-->B01001e25
--women 65+ --> B01001e44-->B01001e49
select GEOID_Data,
       B01001e20+B01001e21+B01001e22+B01001e23+B01001e24+B01001e25+B01001e44+B01001e45+B01001e46+B01001e47+B01001e48+B01001e49 as pop65plus
into #pop
from [dbo].[Census_ACS5Yr_2018]
order by GEOID_Data;

--Step 3: Rurality and SES Characteristics
select [GEOID_Data], [State_Name],[HRSA_RURAL],
case when [Median_ADI_NatRank]<=25 then 2
     when ([Median_ADI_NatRank]>25 and [Median_ADI_NatRank]<=75) then 1
	 when [Median_ADI_NatRank]>75 then 0
	 else 99 end as [ADI]
into #charac
from [dbo].[US_Census_Tract_Charac]

--step 4: build matrix
--Added subquery to reduce tables in code
select GEOID_Data, count(distinct Provider_ID) as hospitals_win_60min, min([Total_TravelTime]) as closest_skilled_nursing
into #key_table
from
(
select A.*, B.pop65plus
FROM [HCAN361_FinalProj].[dbo].[Travel_Matrix] as A
left join #pop as B on A.GEOID_Data=B.GEOID_Data
where Provider_ID in (select ID from #key_ID)
and Total_TravelTime<=60
) as A
group by GEOID_Data
order by GEOID_Data;
--(57581 rows affected)
--some census tracts do not have nursing facilities within our criteria

--Final Table For Map
select  a.GEOID_Data
      , c.State_Name
	  , c.ADI
	  , c.HRSA_RURAL
	  , a.pop65plus
      , isnull(b.hospitals_win_60min,0) as  hospitals_win_60min
	  , isnull(b.closest_skilled_nursing,361) as closest_skilled_nursing
into [HCAN361_25].[dbo].[finalMap]
from #pop as a
left join #key_table as b on A.GEOID_Data=B.GEOID_Data
left join #charac as c on A.GEOID_Data=C.GEOID_Data;

(744 rows affected)

(74001 rows affected)

(74001 rows affected)

(57581 rows affected)

(74001 rows affected)

Total execution time: 00:00:04.329

In [2]:
--Table 1: Median Number of Facilites by Attributes
select *
from
(
select distinct 
'United States' as State_Name
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY hospitals_win_60min)OVER () AS Statewide   
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY case when HRSA_RURAL=0 then hospitals_win_60min end) OVER () AS Urban
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY case when HRSA_RURAL=1 then hospitals_win_60min end) OVER () AS Rural
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY case when ADI=2 then hospitals_win_60min end) OVER () AS [High SES]
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY case when ADI=1 then hospitals_win_60min end) OVER () AS [Middle SES]
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY case when ADI=0 then hospitals_win_60min end) OVER () AS [Low SES]
from [HCAN361_25].[dbo].[finalMap]

--below code concatenates (or stacks tables on top of each other)
union all

select distinct
 State_Name
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY hospitals_win_60min)OVER (PARTITION BY State_Name) AS Statewide   
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY case when HRSA_RURAL=0 then hospitals_win_60min end) OVER (PARTITION BY State_Name) AS Urban
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY case when HRSA_RURAL=1 then hospitals_win_60min end) OVER (PARTITION BY State_Name) AS Rural
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY case when ADI=2 then hospitals_win_60min end) OVER (PARTITION BY State_Name) AS [High SES]
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY case when ADI=1 then hospitals_win_60min end) OVER (PARTITION BY State_Name) AS [Middle SES]
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY case when ADI=0 then hospitals_win_60min end) OVER (PARTITION BY State_Name) AS [Low SES]
from [HCAN361_25].[dbo].[finalMap]

) as A
order by State_Name;


(53 rows affected)

Total execution time: 00:00:05.046

State_Name,Statewide,Urban,Rural,High SES,Middle SES,Low SES
Alabama,1,1,0,1,1,1
Alaska,0,0,0,0,0,0
Arizona,1,1,0,1,1,1
Arkansas,1,2,1,2,2,1
California,5,6,0,6,3,2
Colorado,0,0,0,0,0,0
Connecticut,2,2,1,2,2,2
Delaware,3,5,1,2,4,7
District of Columbia,4,4,NULL,4,4,4
Florida,3,3,0,3,3,3


In [3]:
--Table 2: Median Closest Facility

select *
from
(
select distinct 
'United States' as State_Name
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY closest_skilled_nursing)OVER () AS Statewide   
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY case when HRSA_RURAL=0 then closest_skilled_nursing end) OVER () AS Urban
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY case when HRSA_RURAL=1 then closest_skilled_nursing end) OVER () AS Rural
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY case when ADI=2 then closest_skilled_nursing end) OVER () AS [High SES]
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY case when ADI=1 then closest_skilled_nursing end) OVER () AS [Middle SES]
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY case when ADI=0 then closest_skilled_nursing end) OVER () AS [Low SES]
from [HCAN361_25].[dbo].[finalMap]

--below code concatenates (or stacks tables on top of each other)
union all

select distinct
 State_Name
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY closest_skilled_nursing)OVER (PARTITION BY State_Name) AS Statewide   
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY case when HRSA_RURAL=0 then closest_skilled_nursing end) OVER (PARTITION BY State_Name) AS Urban
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY case when HRSA_RURAL=1 then closest_skilled_nursing end) OVER (PARTITION BY State_Name) AS Rural
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY case when ADI=2 then closest_skilled_nursing end) OVER (PARTITION BY State_Name) AS [High SES]
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY case when ADI=1 then closest_skilled_nursing end) OVER (PARTITION BY State_Name) AS [Middle SES]
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY case when ADI=0 then closest_skilled_nursing end) OVER (PARTITION BY State_Name) AS [Low SES]
from [HCAN361_25].[dbo].[finalMap]

) as A
order by State_Name;

(53 rows affected)

Total execution time: 00:00:05.354

State_Name,Statewide,Urban,Rural,High SES,Middle SES,Low SES
Alabama,56.75472981203249,54.142080230631095,361,58.53533380215538,59.770983896769806,53.29834920072502
Alaska,361,361,361,361,361,361
Arizona,40.24586400275311,34.62100912402852,361,41.27605772417975,39.14376047916538,39.91381767616523
Arkansas,40.2189221090837,34.20212898629481,47.77488905887026,25.283817748033304,38.23987121693436,46.065423435573
California,21.10911668696388,20.05567993530164,361,18.411530559203655,25.32438407844165,26.05516258829263
Colorado,361,361,361,361,361,361
Connecticut,27.31857245727037,25.962780427764727,56.48181691198509,31.113855970896793,23.0582457988058,22.04705622660032
Delaware,31.9163096992091,29.50865028141478,55.4527890390397,34.20579200556283,30.87383592871153,25.50039411950322
District of Columbia,11.84302619342396,11.84302619342396,NULL,11.996196366925455,12.020910414459902,6.972884240655505
Florida,30.879953128067974,29.225122228564842,361,30.260985860685583,30.63659469200887,30.72512786542106
